In [1]:
import pandas as pd
import ast
import json
import re

In [2]:
df = pd.read_csv("C:/Users/cesua/OneDrive/Documentos/Henry/Proyecto_Individual_1/Datasets/movies_dataset.csv")
df

C:\Users\cesua\AppData\Local\Temp\ipykernel_6332\3673067074.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/cesua/OneDrive/Documentos/Henry/Proyecto_Individual_1/Datasets/movies_dataset.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


1. Las columnas belongs_to_collection, production_companies, production_countries y genres están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, se procedera a desanidarlos para poder unirlos al dataset de nuevo, asi poder hacer alguna de las consultas de la API o bien buscar la manera de acceder a esos datos sin desanidarlos.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Proceso de desanidar columna 'belongs_to_collection'

In [4]:
df.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [5]:
# Esta función verifica si el valor proporcionado es un diccionario válido o una cadena que representa un diccionario. 
# Si es un diccionario válido, devuelve el valor correspondiente a la clave especificada, de lo contrario, devuelve None.

def obtener_valor(dicc, clave):
    if pd.isnull(dicc):
        return None
    if not isinstance(dicc, dict):
        try:
            dicc = json.loads(dicc.replace("'", "\""))
        except (json.JSONDecodeError, AttributeError):
            return None
    if isinstance(dicc, dict):
        return dicc.get(clave)
    else:
        return None

In [6]:
# Aplicar la función a la columna "belongs_to_collection"

df['id_btc'] = df['belongs_to_collection'].apply(lambda x: obtener_valor(x, 'id'))
df['name_btc'] = df['belongs_to_collection'].apply(lambda x: obtener_valor(x, 'name'))
df['poster_btc'] = df['belongs_to_collection'].apply(lambda x: obtener_valor(x, 'poster_path'))
df['backdrop_btc'] = df['belongs_to_collection'].apply(lambda x: obtener_valor(x, 'backdrop_path'))

In [ ]:
# Eliminar columna belongs_to_collection

# del df['belongs_to_collection']

In [7]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'id_btc', 'name_btc', 'poster_btc',
       'backdrop_btc'],
      dtype='object')

Proceso de desanidar columna 'genres'

In [8]:
# Convertimos la columna 'genres' de string a lista de diccionarios
df['genres'] = df['genres'].apply(ast.literal_eval)

# Extraemos el nombre de cada genero y creamos una nueva columna con la lista de nombres de generos
df['genre_names'] = df['genres'].apply(lambda x: [d['name'] for d in x])

# Expandimos la lista de nombres de géneros
generos_expandidos = df['genre_names'].explode()

# Contamos la frecuencia de cada valor en la columna 'genre_names'
conteo_generos = generos_expandidos.value_counts()

# Eliminamos los géneros que solo aparecen una vez
conteo_generos = conteo_generos[conteo_generos > 1]

In [32]:
df['genre_names'] = df['genre_names'].astype(str).str.replace('[','').str.replace(']','')

In [34]:
df['genre_names'] = df['genre_names'].str.replace("'", "")

In [ ]:
# Elimina la columna anidada genres
# del df['genres']

Proceso desanidar columna 'production_countries'

In [9]:
# Función personalizada para normalizar la columna "production_countries"
def normalize_production_countries(value):
    try:
        pattern = r"'name':\s+'(.*?)'"
        matches = re.findall(pattern, value)
        return ', '.join(matches)
    except (TypeError, AttributeError):
        return ''

# Aplicamos la función personalizada a cada elemento de la columna "production_countries"
df['country'] = df['production_countries'].apply(normalize_production_countries)

In [10]:
# Eliminamos la columna anidada production_countries
del df['production_countries']

In [11]:
# Renombro la columna a su nombre original
df = df.rename(columns={'country': 'production_countries'})

Proceso desanidar columna 'production_companies'

In [12]:
# Función personalizada para normalizar la columna "production_companies"
def normalize_production_companies(value):
    try:
        pattern = r"'name':\s+'(.*?)'"
        matches = re.findall(pattern, value)
        return [match for match in matches]
    except (TypeError, AttributeError):
        return []

# Aplicamos la función personalizada a cada elemento de la columna "production_companies"
df['production_companies'] = df['production_companies'].apply(normalize_production_companies)

In [13]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'id_btc', 'name_btc',
       'poster_btc', 'backdrop_btc', 'genre_names', 'production_countries'],
      dtype='object')

In [35]:
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
df.head(5)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,id_btc,name_btc,genre_names,production_countries,return,release_year
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,10194.0,Toy Story Collection,"Animation, Comedy, Family",United States of America,12.45,1995.0
1,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,NaN,None,"Adventure, Fantasy, Family",United States of America,4.04,1995.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros., Lancaster Gate",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,119050.0,Grumpy Old Men Collection,"Romance, Comedy",United States of America,0.00,1995.0
3,NaN,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,NaN,None,"Comedy, Drama, Romance",United States of America,5.09,1995.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions, Touchstone Pictures",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,96871.0,Father of the Bride Collection,Comedy,United States of America,0.00,1995.0


In [15]:
df['production_companies'] = df['production_companies'].astype(str).str.replace('[','').str.replace(']','')

In [17]:
df['production_companies'] = df['production_companies'].str.replace("'", "")

2. Los valores nulos de los campos "revenue", "budget" deben ser rellenados por el número 0.

In [19]:
# Verificar el tipo de datos de cada columna
tipos_de_datos = df.dtypes

# Mostrar los tipos de datos de cada columna
print(tipos_de_datos)

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
id_btc                   float64
name_btc                  object
poster_btc                object
backdrop_btc              object
genre_names               object
production_countries      object
dtype: object


In [20]:
# Los valores nulos de los campos revenue, budget son rellenados por el número 0
df['revenue'].fillna(0, inplace=True)
df['budget'].fillna(0, inplace=True)
print('Nulos en revenue',df.revenue.isna().sum())
print('Nulos en budget',df.revenue.isna().sum())

Nulos en revenue 0
Nulos en budget 0


In [21]:
# Reemplazar los valores no numéricos por NaN
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

In [22]:
# convierte la columna budget en flotante
df['budget'] = df['budget'].astype(float)

3. Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [23]:
# Definir una función para calcular el retorno de inversión
def calcular_retorno(row):
    revenue = row['revenue']
    budget = row['budget']
    
    # Verificar si hay datos faltantes o el presupuesto es igual a cero
    if pd.isnull(revenue) or pd.isnull(budget) or budget == 0:
        return 0
    
    # Calcular el retorno de inversión y redondearlo a dos decimales
    return round(revenue / budget, 2)

# Aplicar la función a cada fila del DataFrame para crear la columna "return"
df['return'] = df.apply(calcular_retorno, axis=1)

In [24]:
df['return'].isna().sum()

0

In [25]:
df['return'].unique()

array([12.45,  4.04,  0.  , ..., 12.75, 24.39, 16.8 ])

4. Los valores nulos del campo release date deben eliminarse.

In [26]:
# eliminamos los registros con nulos 
df = df.dropna(subset = ['release_date'])

# controlamos para verificar que no existan nulos
df['release_date'].isnull().sum()

0

5. De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [27]:
# De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.
# Le damos el formato de fecha a todos los valores de la columna y los que no, le asignamos la etiqueta de nulo
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Se asegura que todos los registros tengan el mismo formato de fecha AAAA-MM-DD
df['release_date'] = pd.to_datetime(df['release_date']).dt.strftime('%Y-%m-%d')

# se identifican la cantidad de datos nulos de la columna 'release_date' 
df['release_date'].isnull().sum()

3

In [28]:
# creamos una nueva columna 'release_year' que va a contener unicamnete el dato de año de la columna 'release_date' 
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_year']=df['release_date'].dt.year

# hacemos control sobre los cambios
df[['release_date','release_year']].head(10)

,release_date,release_year
0,1995-10-30,1995.0
1,1995-12-15,1995.0
2,1995-12-22,1995.0
3,1995-12-22,1995.0
4,1995-02-10,1995.0
5,1995-12-15,1995.0
6,1995-12-15,1995.0
7,1995-12-22,1995.0
8,1995-12-22,1995.0
9,1995-11-16,1995.0


6. Eliminar las columnas que no serán utilizadas, video, imdb_id, adult, original_title, homepage y poster_path.

In [29]:
# Se eliminan las columnas inecesarias
df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'vote_count', 'poster_path', 'homepage'], inplace = True)
df.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'id_btc', 'name_btc', 'poster_btc', 'backdrop_btc',
       'genre_names', 'production_countries', 'return', 'release_year'],
      dtype='object')

In [30]:
# Se eliminan columnas adicional que no tienen relevancio en el dataframe como poster_btc, backdrop_btc
df.drop(columns=['poster_btc', 'backdrop_btc'], inplace = True)

In [36]:
df.head(10)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,id_btc,name_btc,genre_names,production_countries,return,release_year
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,10194.0,Toy Story Collection,"Animation, Comedy, Family",United States of America,12.45,1995.0
1,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,NaN,None,"Adventure, Fantasy, Family",United States of America,4.04,1995.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros., Lancaster Gate",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,119050.0,Grumpy Old Men Collection,"Romance, Comedy",United States of America,0.00,1995.0
3,NaN,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,NaN,None,"Comedy, Drama, Romance",United States of America,5.09,1995.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions, Touchstone Pictures",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,96871.0,Father of the Bride Collection,Comedy,United States of America,0.00,1995.0
5,NaN,60000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",949,en,"Obsessive master thief, Neil McCauley leads a ...",17.924927,"Regency Enterprises, Forward Pass, Warner Bros.",1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,7.7,NaN,None,"Action, Crime, Drama, Thriller",United States of America,3.12,1995.0
6,NaN,58000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",11860,en,An ugly duckling having undergone a remarkable...,6.677277,"Paramount Pictures, Scott Rudin Productions, M...",1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,6.2,NaN,None,"Comedy, Romance","Germany, United States of America",0.00,1995.0
7,NaN,0.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",45325,en,"A mischievous young boy, Tom Sawyer, witnesses...",2.561161,Walt Disney Pictures,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,5.4,NaN,None,"Action, Adventure, Drama, Family",United States of America,0.00,1995.0
8,NaN,35000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",9091,en,International action superstar Jean Claude Van...,5.23158,"Universal Pictures, Imperial Entertainment, Si...",1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,5.5,NaN,None,"Action, Adventure, Thriller",United States of America,1.84,1995.0
9,"{'id': 645, 'name': 'James Bond Collection', '...",58000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 2

In [37]:
df.to_csv('C:/Users/cesua/OneDrive/Documentos/Henry/Proyecto_Individual_1/Datasets/movies_dataset_final.csv', index=False)